# BGT zadanie 4
## Autors
- Paweł Benkowski
- Stanisław Lutkiewicz"
### Task
Downloads Nietzsche and freud books count words in it.  
Finds top 10 words and 3 top nouns  

In [8]:
import sys
!{sys.executable} -m pip install PyPDF2
!{sys.executable} -m pip install requests
!{sys.executable} -m pip install bs4
!{sys.executable} -m pip install nltk

In [9]:
from pyspark import SparkContext, SparkFiles
from pathlib import Path
from bs4 import BeautifulSoup
from collections import Counter
from operator import add
import os
import requests
import PyPDF2
import re
import nltk
import timeit
nltk.download("averaged_perceptron_tagger")

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [10]:
def get_books_urls(url):
    urlsWithName = []
    r = requests.get(url)
    soup = BeautifulSoup(r.text, "html.parser")
    
    for a in soup.find_all("a", href=True):
        arr = a["href"].split(".")
        if arr[-1] == "pdf":
            urlsWithName.append(("http:" +a["href"],a.string))
    return urlsWithName

In [11]:
def get_books_and_create_folder(urls, target):
    if not os.path.isdir(target):
        os.makedirs(target)
    for url in urls:
        file = Path(target,url[1] + ".pdf")
        if not file.exists ():
            response = requests.get(url[0])
            with open(Path(target, url[1] + ".pdf"), "wb+") as f:
                f.write(response.content)
        else:
            print("File already exists")


In [12]:
def save_text_from_pdf(name):
    file = Path(name + ".txt")
    if not file.exists ():
        fileReader = PyPDF2.PdfFileReader(name)
        with open(Path(name + ".txt"), "a+") as f:
            for page in fileReader.pages:
                f.write(page.extractText())

In [13]:
def is_noun(word : str) -> bool:
    if "s" == word or "i" == word:
        return False
    partOfSpeech = nltk.pos_tag([word])
    if "NN" == partOfSpeech[0][1]:
        return True
    return False

In [20]:
def compose_text_files(textFiles):
    if len(textFiles) == 1 :
        return textFiles[0].name
    with open(textFiles[0],'a') as toAppend:
        for txt in textFiles[1::]:
            with open(txt) as fromAppend:
                toAppend.write(fromAppend.read())
            os.remove(txt)
    return textFiles[0].name

In [16]:
sc = SparkContext("local", "first app")

In [21]:
folderNamesWithUrls = [('freud',"https://holybooks.com/sigmund-freud-the-complete-works/"),\
    ('nitsche',"https://holybooks.com/the-complete-works-of-friedrich-nietzsche-in-english-as-pdf/")]
files = []
for fnwu in folderNamesWithUrls:
    urls = get_books_urls(fnwu[1])
    get_books_and_create_folder(urls, fnwu[0])
    pdfFiles = list(Path(fnwu[0]).glob('*.pdf'))
    for pdf in pdfFiles:
        save_text_from_pdf(os.path.join(fnwu[0], pdf.name))
    textFileName = compose_text_files(list(Path(fnwu[0]).glob('*.txt')))
    files.append(sc.textFile(os.path.join(fnwu[0], textFileName)).cache())

File already exists
File already exists
File already exists
File already exists
File already exists
File already exists
File already exists
File already exists
File already exists
File already exists
File already exists
File already exists
File already exists
File already exists
File already exists
File already exists
File already exists
File already exists
File already exists


In [30]:
start = timeit.default_timer()
for file in files:
    startIn = timeit.default_timer()
    print(file)
    lowerCaseText = file.map(lambda x: x.lower())
    words = lowerCaseText.map(lambda x : re.findall(r"\b[^\d\W]+\b", x)).flatMap(lambda x: x)
   
    wordsForCounting = words.map(lambda x: (x,1))
    countedNouns = wordsForCounting.reduceByKey(add).filter(lambda x: is_noun(x[0])).collect()
    
    kNouns = Counter(dict(countedNouns))
    kWords = Counter(words.collect())
    
    print(f"total word {words.count()}")
    print(f"most common {kWords.most_common(10)}")
    print(f"top nouns {kNouns.most_common(3)}")
    print(f"time spend {timeit.default_timer() - startIn}")
    print(30*"----")
print(f"total time spend {timeit.default_timer()-start}")

freud/Sigmund Freud – The Complete Works.pdf.txt MapPartitionsRDD[5] at textFile at NativeMethodAccessorImpl.java:0
total word 2068906
most common [('the', 142371), ('of', 94924), ('to', 60988), ('in', 50950), ('a', 48808), ('and', 46208), ('that', 33805), ('is', 30431), ('it', 27788), ('which', 22649)]
top nouns [('dream', 6115), ('analysis', 3584), ('time', 3456)]
time spend 17.891764156986028
------------------------------------------------------------------------------------------
nitsche/The complete works of Friedrich Nietzsche VOL XVI.pdf.txt MapPartitionsRDD[7] at textFile at NativeMethodAccessorImpl.java:0
total word 1304489
most common [('the', 84034), ('of', 59756), ('and', 44130), ('to', 36902), ('a', 26105), ('in', 24907), ('is', 21215), ('that', 16802), ('it', 14520), ('as', 13386)]
top nouns [('man', 4285), ('life', 2701), ('power', 2536)]
time spend 17.786092587019084
------------------------------------------------------------------------------------------
total time s